In [ ]:
import time
from tqdm import tqdm
import csv
import json
import numpy as np
import argparse

subscription_key =  #input subscription key
assert subscription_key
cx = #input cx
assert cx

search_url = "https://www.googleapis.com/customsearch/v1?"
import requests

headers={"key": subscription_key, "cx": cx}

from IPython.display import HTML

def grab_web_text(search_query, max_sentences):
    params = {"key": subscription_key, "cx": cx, "q": search_query.lower(), "num": max_sentences}
    response = requests.get(search_url, params=params)
    response.raise_for_status()
    search_results = response.json()
    urls = []
    snippets = []
    titles = []
    if "items" not in search_results:
        snippets = [search_query] * max_sentences
        return urls, snippets
    for v in search_results["items"]:
        if not isinstance(v['link'], str):
            v['link'] = v['link'].decode('utf-8')
        if not isinstance(v['title'], str):
            v['title'] = v['title'].decode('utf-8')

        urls.append(v['link'])
        titles.append(v['title'])

        if not isinstance(v["snippet"],str):
            v["snippet"] = v["snippet"].encode("utf-8")
        #if "datePublished" in v:
        #    print (v["datePublished"])

        snippets.append(v["snippet"].replace("<b>","").replace("</b>","").replace("\n",""))
        
    
    return urls, snippets, titles

def live_crawl(id, query, limit):
    print ('Querying: {} ...'.format(query))

    js = {}
    urls, snippets, titles = grab_web_text(query, limit)

    js['id'] = id
    js['claim'] = query.lower()
    js['snippets'] = snippets
    js['titles'] = titles
    js['urls'] = urls

    return js

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--query_path', type=str, default='input.txt')
    parser.add_argument('--output_path', type=str, default='output.json')
    parser.add_argument('--limit', type=int, default=10)
    args = parser.parse_args()
    
    with open(args.query_path, 'r') as f:
        query = f.read().splitlines()
    
    count=0
    all_records = []
    for q in query:
        js = live_crawl(count, q, args.limit)
        all_records.append(js)
        count += 1

    records = {'records': all_records}
    with open(args.output_path, 'w+') as g:
        g.write(json.dumps(records)